In [ ]:
import numpy as np
import pylab as plt
import svgwrite
from skimage import io, filters, measure, feature, exposure, morphology
from scipy import ndimage, spatial
from scipy.ndimage import rotate
import os
import random
from IPython.display import SVG
%matplotlib inline

In [ ]:
ao_monkey = io.imread('TestMonkey/AO.png')
uv_monkey = io.imread('TestMonkey/UV.png')
normal_monkey = io.imread('TestMonkey/Normal.png')
original_monkey = io.imread('TestMonkey/Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0])
coords = feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=5)
print(coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],"ro")
plt.imshow(edge_monkey)

In [ ]:
def center_angle(p1, p2):
    x_difference = p2[1]-p1[1]
    y_difference = p2[0]-p1[0]
    return np.rad2deg(np.arctan2(y_difference, x_difference))

In [ ]:
mean_x = np.mean(coords[1,:])
mean_y = np.mean(coords[0,:])
mean_point = [mean_y, mean_x]

start = coords[np.where(spatial.distance.cdist([[0, 0]], coords).max())]
angles = []
for i in coords:
    angles.append(center_angle(i, mean_point))

angles = np.asarray(angles)
coords = coords[angles.argsort()]
print(start, coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],)
plt.imshow(edge_monkey)

In [ ]:
dwg = svgwrite.Drawing('stroke.svg')

dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

for i in range(0,10):
    genid = chr(i+48)
    gradline = dwg.linearGradient((0, 0), (1, 0), id=genid)
    gradline.add_stop_color(1, 'white', 0)
    gradline.add_stop_color(round(random.uniform(0.85, 0.95), 2), 'black')

    dwg.defs.add(gradline)
    
    start_x = random.randint(10, 100)
    start_y = random.randint(10, 100)
    
    dwg.add(dwg.line((start_x, start_y),
                     (start_x + random.randint(25, 100), start_y + random.randint(25, 100)),
                     stroke="url(#" + genid + ")",
                     stroke_linecap="round"))

display(SVG(dwg.tostring().replace('><','>\n<')))

dwg.save()

In [ ]:
def sketch(points, name):
    precision = 5
    dwg = svgwrite.Drawing(name)
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))
    
    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    #gradline.add_stop_color(1, 'white', 0.1)
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    for i in range(1, len(points)):
        dwg.add(dwg.line((round(points[i-1][0], precision),(round(points[i-1][1], precision))),
                         ((round(points[i][0], precision), (round(points[i][1], precision)))),
                         stroke="url(#grad)",
                         stroke_linecap="round"))

    display(SVG(dwg.tostring().replace('><','>\n<')))

    return dwg

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0] + normal_monkey[:,:,0])
new_edge_monkey =np.flip(np.rot90(edge_monkey), axis=0)
contours = measure.find_contours(new_edge_monkey, 0.17, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
svg = sketch(simplified_contours[0], 'sketch.svg')
svg.save()

In [ ]:
def path_sketches(points):
    precision = 5
    dwg = svgwrite.Drawing('path1.svg')

    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    gradline.add_stop_color(1, 'white', 0.8)
    gradline.add_stop_color(0.9, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0][0,0], points[0][0,1]), fill='black', stroke='black')
    for pointSet in points:
        path.push('M', pointSet[0,0], pointSet[0,1])
        for p in pointSet[1:]:
            path.push(('L', p[0], p[1]))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketches(simplified_contours)
path_out.save()

In [ ]:
def angle(p1):
    ang = np.arctan2(p1[0], p1[1])
    return np.rad2deg((ang) % (2 * np.pi))

In [ ]:
def path_sketch(points, corners=[]):
    corners[:,[0,1]] = corners[:,[1,0]]
    dwg = svgwrite.Drawing('path1.svg')
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black')
    for i in range(1, len(points)):
        path.push(('L', points[i,0], points[i,1]))
        print(points[i-1], points[i])
        if points[i] in corners:
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i-1,0])*2.5, points[i,1]+(points[i,1]-points[i-1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
            
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i+1,0])*2.5, points[i,1]+(points[i,1]-points[i+1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketch(simplified_contours[0], feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=20))
path_out.save()

In [ ]:
edge_monkey = filters.sobel(normal_monkey[:,:,0]) + filters.sobel(ao_monkey[:,:,0])
bin_edge_monkey = np.where(edge_monkey<0.1, 0, 1)
plt.figure(figsize=(20,20))
plt.imshow(ndimage.gaussian_filter(bin_edge_monkey, sigma=0.12), cmap = plt.cm.Greys)

In [ ]:
def momentum_sketch(points):
    velocity = 0.1
    dwg = svgwrite.Drawing('momentum.svg')
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = 0.1
        currentPos = np.copy(points[i-1])
        print(currentPos)
        
        angle = np.rad2deg(np.arctan2([points[i-1][1], points[i][1]], [points[i-1][0], points[i][0]]))
        print(angle)
        while lessThan(points[i-1], currentPos, points[i]):
            currentPos = [currentPos[0]+( velocity*np.cos(angle[0]) ), currentPos[1]+( velocity*np.sin(angle[1]) )]
            print(currentPos)
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + 0.2
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def lessThan(startPos, currentPos, endPos):
    if np.linalg.norm(currentPos-startPos) < np.linalg.norm(endPos-startPos):
        return True
    else:
        return False

In [ ]:
path_out = momentum_sketch(simplified_contours[0])
path_out.save()